<a href="https://colab.research.google.com/github/VindhyaSRajan/Master-s-Thesis/blob/master/Nested_XML_to_pandasDATAFRAME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Mounting the drive


In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [117]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Import Section

In [0]:
import pandas as pd
import xml.etree.ElementTree as et
import os
from pathlib import Path
import glob

##Reading the nested XML file into pandas dataframe


In [0]:
with os.scandir('/content/drive/My Drive/NLMCXR_reports/ecgen-radiology/') as entries:
    for entry in entries:
        print(entry.name)

In [0]:
print(glob.glob("/content/drive/My Drive/NLMCXR_reports/ecgen-radiology/*.xml")) #Run this to check Colab is reading the folder

###Reading the child nodes of e-Citation, calling it the LEFT Dataframe

In [120]:
def getvalueofnode(node):
    """ return node text or None """
    return node.text if node is not None else None

def main():
    """ main """
    filenames =  glob.glob("/content/drive/My Drive/NLMCXR_reports/ecgen-radiology/*.xml") #Change to your filepath
    filename = [] 
    li = []
    for filename in filenames :
         
         global left
         parsed_xml = et.parse(filename)
         dfcols = ['uId', 'pmcId', 'Owner', 'parentImage_id']
         left = pd.DataFrame(columns=dfcols)

         root = parsed_xml.getroot()
         
        
         for uIdFind in parsed_xml.findall('./uId'):
             uId = uIdFind.get('id')

         for pmcIdFind in parsed_xml.findall('./pmcId'):
             pmcId = pmcIdFind.get('id')
    
         for MedlineCitationFind in parsed_xml.findall('./MedlineCitation'):
             Owner = MedlineCitationFind.get('Owner')
       
         for parentImage_idFind in parsed_xml.findall('./parentImage'):
             ParentImageId = parentImage_idFind.get('id')
             
             left = left.append(pd.Series([uId, pmcId, Owner, ParentImageId], index=dfcols),ignore_index=True)
             li.append(left)
             
    left = pd.concat(li, axis=0, ignore_index=True)

    return left

main()

left = main()

left.head(5)

    
#     BidIdentificationString = './AllocationTimeSeries[%d]/BidIdentification'%(i) -- DO NOT DELETE THIS LINE
    
    

,uId,pmcId,Owner,parentImage_id
0,CXR1158,1158,Indiana University,CXR1158_IM-0107-1001
1,CXR1158,1158,Indiana University,CXR1158_IM-0107-1001
2,CXR1158,1158,Indiana University,CXR1158_IM-0107-2001
3,CXR2821,2821,Indiana University,CXR2821_IM-1244-1001
4,CXR2821,2821,Indiana University,CXR2821_IM-1244-1001


In [121]:
left.shape #Shape should be (11321,4)

(11321, 4)

In [122]:
left = left.drop_duplicates()
print('Shape after dropping all duplicate values of the left datatframe : ', left.shape) #Shape should be (7470,4)
left.head(5)

Shape after dropping all duplicate values of the left datatframe :  (7470, 4)


,uId,pmcId,Owner,parentImage_id
0,CXR1158,1158,Indiana University,CXR1158_IM-0107-1001
2,CXR1158,1158,Indiana University,CXR1158_IM-0107-2001
3,CXR2821,2821,Indiana University,CXR2821_IM-1244-1001
5,CXR2821,2821,Indiana University,CXR2821_IM-1244-2001
6,CXR3005,3005,Indiana University,CXR3005_IM-1388-1001


In [123]:
left.iloc[0] #Ensuring that the results are aligned with the original XML and dataframe

uId                            CXR1158
pmcId                             1158
Owner               Indiana University
parentImage_id    CXR1158_IM-0107-1001
Name: 0, dtype: object

###Reading the heavily nested elements, calling it the RIGHT Dataframe

In [0]:
filenames = glob.glob("/content/drive/My Drive/NLMCXR_reports/ecgen-radiology/*.xml") #Change to your file path
print(filenames)

In [134]:
def rightmain():
    """ main """
    filenames =  glob.glob("/content/drive/My Drive/NLMCXR_reports/ecgen-radiology/*.xml") #Change to your filepath
    filename = [] 
    li = []
    for filename in filenames :
         
         global right
         parsed_xml = et.parse(filename)
         dfcols_right = ['uId', 'Comparison', 'Indication', 'Findings', 'Impression', 'Major']
         right = pd.DataFrame(columns= dfcols_right)

         root = parsed_xml.getroot()
         
        
         for uIdFind in parsed_xml.findall('./uId'):
             uId = uIdFind.get('id')
         Comparison = (root[16][0][2][0].text) #This is how to access the Comparison
         Indication = (root[16][0][2][1].text) #This is how to access the Indication
         Findings = (root[16][0][2][2].text) #This is how to access the Findings
         Impression = (root[16][0][2][3].text) #This is how to access the Impression
         Major = (root[17][0].text) #This is how to access the Major
         #Major_2 = (root[17][1].text) #This is how to access the Major
            
         right = right.append(pd.Series([uId,Comparison,Indication,Findings,Impression,Major], index=dfcols_right),ignore_index=True)
         li.append(right)
             
    right = pd.concat(li, axis=0, ignore_index=True)

    return right

rightmain()

right = rightmain()

right.head(5)

,uId,Comparison,Indication,Findings,Impression,Major
0,CXR1158,"XXXX, XXXX.",dyspnea,Heart size remains slightly large. Aorta remai...,Continued slight cardiomegaly with no evidence...,Cardiomegaly/mild
1,CXR2821,None,Right-sided chest pain after XXXX,Cardiomediastinal silhouette and pulmonary vas...,No acute cardiopulmonary findings.,Calcinosis/aorta
2,CXR3005,XXXX,XXXX with XXXX and XXXX loss,The heart is normal in size. The mediastinum i...,1. Emphysema without acute infiltrate. 2. Prog...,Aorta/tortuous
3,CXR3899,"CT chest XXXX, XXXX and chest radiograph XXXX,...",XXXX-year-old with abdominal pain.,Chest. Mildly hyperexpanded lungs. The right l...,Chest 1. No acute cardiopulmonary findings. Ab...,Lung/hyperdistention/mild
4,CXR2667,None available.,"XXXX-year-old male with XXXX, XXXX, and XXXX.",Heart size normal. No focal airspace disease. ...,No acute cardiopulmonary findings.,normal


In [135]:
right.shape #Shape should be (3955,6) as we have 3955 reports

(3955, 6)

In [136]:
right.columns

Index(['uId', 'Comparison', 'Indication', 'Findings', 'Impression', 'Major'], dtype='object')

##Merging the two dataframes : LEFT and RIGHT

In [139]:
result = pd.merge(left, right, how='outer', on=['uId'])
print('The shape for the final merged data frame is: ',result.shape) #should be (7574,9)
result.head()

The shape for the final merged data frame is:  (7574, 9)


,uId,pmcId,Owner,parentImage_id,Comparison,Indication,Findings,Impression,Major
0,CXR1158,1158,Indiana University,CXR1158_IM-0107-1001,"XXXX, XXXX.",dyspnea,Heart size remains slightly large. Aorta remai...,Continued slight cardiomegaly with no evidence...,Cardiomegaly/mild
1,CXR1158,1158,Indiana University,CXR1158_IM-0107-2001,"XXXX, XXXX.",dyspnea,Heart size remains slightly large. Aorta remai...,Continued slight cardiomegaly with no evidence...,Cardiomegaly/mild
2,CXR2821,2821,Indiana University,CXR2821_IM-1244-1001,None,Right-sided chest pain after XXXX,Cardiomediastinal silhouette and pulmonary vas...,No acute cardiopulmonary findings.,Calcinosis/aorta
3,CXR2821,2821,Indiana University,CXR2821_IM-1244-2001,None,Right-sided chest pain after XXXX,Cardiomediastinal silhouette and pulmonary vas...,No acute cardiopulmonary findings.,Calcinosis/aorta
4,CXR3005,3005,Indiana University,CXR3005_IM-1388-1001,XXXX,XXXX with XXXX and XXXX loss,The heart is normal in size. The mediastinum i...,1. Emphysema without acute infiltrate. 2. Prog...,Aorta/tortuous


In [150]:
type(result)

pandas.core.frame.DataFrame

##Converting the final merged dataframe into a CSV and saving to the Drive

In [0]:
from google.colab import files
result.to_csv('OpeniReports.csv')

##Data Analysis of the Open-i Reports

In [156]:
##Filtering out all rows with value as None
is_filtered_df = result['Comparison'] != 'None'
print(is_filtered_df.head())
filtered_df = result[is_filtered_df]
filtered_df.head()

0     True
1     True
2    False
3    False
4     True
Name: Comparison, dtype: bool


,uId,pmcId,Owner,parentImage_id,Comparison,Indication,Findings,Impression,Major
0,CXR1158,1158,Indiana University,CXR1158_IM-0107-1001,"XXXX, XXXX.",dyspnea,Heart size remains slightly large. Aorta remai...,Continued slight cardiomegaly with no evidence...,Cardiomegaly/mild
1,CXR1158,1158,Indiana University,CXR1158_IM-0107-2001,"XXXX, XXXX.",dyspnea,Heart size remains slightly large. Aorta remai...,Continued slight cardiomegaly with no evidence...,Cardiomegaly/mild
4,CXR3005,3005,Indiana University,CXR3005_IM-1388-1001,XXXX,XXXX with XXXX and XXXX loss,The heart is normal in size. The mediastinum i...,1. Emphysema without acute infiltrate. 2. Prog...,Aorta/tortuous
5,CXR3899,3899,Indiana University,CXR3899_IM-1978-1001,"CT chest XXXX, XXXX and chest radiograph XXXX,...",XXXX-year-old with abdominal pain.,Chest. Mildly hyperexpanded lungs. The right l...,Chest 1. No acute cardiopulmonary findings. Ab...,Lung/hyperdistention/mild
6,CXR2667,2667,Indiana University,CXR2667_IM-1146-1001,None available.,"XXXX-year-old male with XXXX, XXXX, and XXXX.",Heart size normal. No focal airspace disease. ...,No acute cardiopulmonary findings.,normal


In [157]:
filtered_df.shape

(6462, 9)

##This was the initial code block to test how to extract all elements from the XML file

In [0]:
tree = et.parse(filename)
root = tree.getroot()
root.tag
root.attrib
for child in root:
    print(child.tag, child.attrib)

parsed_xml = et.parse("/content/drive/My Drive/NLMCXR_reports/ecgen-radiology/607.xml")
for uIdFind in parsed_xml.findall('./uId'):
  uId = uIdFind.get('id')
root = tree.getroot()
Comparison = (root[16][0][2][0].text) #This is how to access the Comparison
Indication = (root[16][0][2][1].text) #This is how to access the Indication
Findings = (root[16][0][2][2].text) #This is how to access the Findings
Impression = (root[16][0][2][3].text) #This is how to access the Impression
Major_1 = (root[17][0].text) #This is how to access the Major
Major_2 = (root[17][1].text) #This is how to access the Major
print(uId,Comparison,Indication,Findings,Impression,Major_1,Major_2)

CXR607 PA and lateral chest radiographs XXXX. XXXX-year-old female, lung nodules. The cardiomediastinal silhouette is within normal limits for size and contour. The lungs are normally inflated without evidence of focal airspace disease, pleural effusion, or pneumothorax. Scattered bilateral calcified pulmonary nodules. No acute bone abnormality. No acute cardiopulmonary process. Calcinosis/lung/bilateral/scattered Nodule/lung/bilateral/scattered


In [0]:
d = {'uId': uId,'Comparison': (root[16][0][2][0].text), 'Indication': (root[16][0][2][1].text), 'Findings': (root[16][0][2][2].text), 'Impression' : (root[16][0][2][3].text), 'Major_1' : (root[17][0].text), 'Major_2' : (root[17][1].text)}
right = pd.DataFrame(data=d,index =['0'])
right.head()
right.columns

Index(['uId', 'Comparison', 'Indication', 'Findings', 'Impression', 'Major_1',
       'Major_2'],
      dtype='object')

##Miscellaneous Code -- To be discarded later

In [0]:
xtree = et.parse("/content/drive/My Drive/NLMCXR_reports/ecgen-radiology/607.xml")

In [0]:
for node in xroot: 
    abstract_comparison = node.attrib.get("COMPARISON")
    abstract_indication = node.attrib.get("INDICATION")
    abstract_findings = node.attrib.get("label")
    abstract_impression = node.attrib.get("IMPRESSION")
    rows.append({"COMPARISON": abstract_comparison, "INDICATION": abstract_indication, 
                 "FINDINGS": abstract_findings, "IMPRESSION": abstract_impression})


In [0]:
xroot = xtree.getroot()
rows = []

In [0]:
out_df.head(2)

,COMPARISON,INDICATION,FINDINGS,IMPRESSION
0,None,None,None,None
1,None,None,None,None


In [0]:
def parse_XML(xml_file, df_cols): 
    """Parse the input XML file and store the result in a pandas 
    DataFrame with the given columns. 
    
    The first element of df_cols is supposed to be the identifier 
    variable, which is an attribute of each node element in the 
    XML data; other features will be parsed from the text content 
    of each sub-element. 
    """
    
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    
    for node in xroot: 
        res = []
        res.append(node.attrib.get(df_cols[0]))
        for el in df_cols[1:]: 
            if node is not None and node.find(el) is not None:
                res.append(node.find(el).text)
            else: 
                res.append(None)
        rows.append({df_cols[i]: res[i] 
                     for i, _ in enumerate(df_cols)})
    
    out2_df = pd.DataFrame(rows, columns=df_cols)
        
    return out2_df

In [0]:
df_cols = ["COMPARISON", "INDICATION", "FINDINGS", "IMPRESSION"]
out_df = pd.DataFrame(rows, columns = df_cols)

In [0]:
import xml.etree.ElementTree as ET
import csv

tree = ET.parse("/content/drive/My Drive/NLMCXR_reports/ecgen-radiology/1303.xml")
root = tree.getroot()

# open a file for writing

Resident_data = open('/content/drive/My Drive/NLMCXR_reports/ecgen-radiology/1303.csv', 'w')

# create the csv writer object

csvwriter = csv.writer(Resident_data)
resident_head = []

count = 0
for member in root.findall('AbstractText'):
	abstract = []
	#address_list = []
	if count == 0:
		comparison = member.find('COMPARISON').tag
		resident_head.append(comparison)
		indication = member.find('INDICATION').tag
		resident_head.append(indication)
		findings = member.find('FINDINGS').tag
		resident_head.append(findings)
		impression = member.find('IMPRESSION').tag
		resident_head.append(impression)
		csvwriter.writerow(resident_head)
		count = count + 1

	comparison = member.find('COMPARISON').text
	abstract.append(comparison)
	indication = member.find('INDICATION').text
	abstract.append(indication)
	findings = member.find('FINDINGS').text
	abstract.append(findings)
	impression = member.find('IMPRESSION').text
	abstract.append(impression)
	#City = member[3][1].text
	#address_list.append(City)
	#StateCode = member[3][2].text
	#address_list.append(StateCode)
	#PostalCode = member[3][3].text
	#address_list.append(PostalCode)
	#resident.append(address_list)
	csvwriter.writerow(resident)
Resident_data.close()

##Trying to convert the xml file into JSON


In [0]:
import json
import pandas as pd
from pandas.io.json import json_normalize 
with open('/content/drive/My Drive/Data_JSON/999.json', 'r') as f:
    data = json.load(f)
df_json = pd.DataFrame(data)
df_json.transpose()
df_norm_json = json_normalize(data 
                    )
df_norm_json.head()
df_norm_json.columns
df_norm_json.drop(['eCitation.IUXRId._id', 'eCitation.MeSH.major',
       
       'eCitation.MedlineCitation.Article.Affiliation',
       'eCitation.MedlineCitation.Article.ArticleDate.Day',
       'eCitation.MedlineCitation.Article.ArticleDate.Month',
       'eCitation.MedlineCitation.Article.ArticleDate.Year',
       'eCitation.MedlineCitation.Article.ArticleTitle',
       'eCitation.MedlineCitation.Article.AuthorList.Author',
       'eCitation.MedlineCitation.Article.AuthorList._CompleteYN',
       'eCitation.MedlineCitation.Article.Journal.JournalIssue.PubDate.Day',
       'eCitation.MedlineCitation.Article.Journal.JournalIssue.PubDate.Month',
       'eCitation.MedlineCitation.Article.Journal.JournalIssue.PubDate.Year',
       'eCitation.MedlineCitation.Article.Language',
       'eCitation.MedlineCitation.Article.PublicationTypeList.PublicationType',
       'eCitation.MedlineCitation.Article._PubModel',
       'eCitation.MedlineCitation.EssieArticleTitle',
       'eCitation.MedlineCitation.IMedAuthor',
       'eCitation.MedlineCitation._Owner', 'eCitation.MedlineCitation._Status',
       'eCitation.articleDate', 'eCitation.articleType',
       'eCitation.articleURL', 'eCitation.ccLicense', 'eCitation.docSource',
       'eCitation.licenseType', 'eCitation.licenseURL', 'eCitation.meta._type',
       
       'eCitation.title', 'eCitation.uId._id'])
df_norm_json.head()